# Proyecto Final Bootcamp. Primera Parte

El proyecto consiste en un RAG que permitira obtener información precisa sobre datos historicos relacionados con la migracion de España a latinoamerica.

La primera version se enfoca en lo explicado en:
- https://machinelearningmastery.com/building-a-rag-pipeline-with-llama-cpp-in-python/
- https://github.com/pixegami/langchain-rag-tutorial

A continuación se explicara paso a paso la creacion de la primera version de nuestro RAG:

## 1) Instalar librerias requeridas

Lista de librerias:
- OpenAI: Cargar y ejecutar modelos LLM (Large Language Models) de OpenAI.
- Langchain: Gestionar la carga, división, vectorización y consulta de documentos con el LLM.
- Chromadb: Guarda la información de los pdfs de una forma que los LLMs lo puedan entender.
- Unstructured pdf: Extrae el contenido (texto y estructura) de archivos PDF de forma precisa, permitiendo convertir documentos complejos en fragmentos de texto útiles para los LLMs.

In [1]:
%%capture
# Instalar librerias para construir el RAG
!pip install --upgrade langchain langchain-community langchain-openai langchain-chroma chromadb openai "unstructured[pdf]"
!apt-get install -y poppler-utils

## 2) Importar librerias

In [2]:
# Importar librerias
# - Miscelaneos
import os
import time
from pathlib import Path
from tqdm import tqdm
from dotenv import load_dotenv
import textwrap
# - Operar documentos (Leer)
from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
# - Operar documentos (Dividir en bloques)
from langchain.text_splitter import RecursiveCharacterTextSplitter
# - Operar documentos (transformar con embeddings)
from langchain_openai import OpenAIEmbeddings
# - Operar documentos (Guardar)
from langchain_chroma import Chroma
# - Configurar LLMs
from langchain_openai import ChatOpenAI
# - Configurar RAG
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader

## X) Load Drive

In [3]:
## Load Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Definir ruta donde se encuentra el projecto
ruta_principal = "/content/drive/My Drive/Colab Notebooks/PROYECTO FINAL"

In [5]:
# Cargar variables de entorno
load_dotenv(dotenv_path=Path(ruta_principal)/".env")

True

## 3) Operar los documentos

En este paso se leeran y operaran los documentos que se usaran para darle contexto al LLM, es decir para darle informacion precisa al RAG.

### 3.1) Cargar documentos

In [6]:
# Define ruta donde se encontraran los libros
ruta_libros = Path(ruta_principal)/"Libros"

In [7]:
# Cargar los libros
print(f"Cargadon documento de la ruta: {ruta_libros} usando UnstructuredPDFLoader...")
# Inicializar lista donde se cargan los documentos
documents = []
try:
    loader = DirectoryLoader(
        ruta_libros,
        glob="**/*.pdf",
        loader_cls=UnstructuredPDFLoader, # <--- Necesario para leer objetos como tablas.
        show_progress=True
    )
    documents = loader.load()
    print(f"\n{len(documents)} documentos correctamente cargados.")
except Exception as e:
    print(f"Error cargando archivos: {e}")

Cargadon documento de la ruta: /content/drive/My Drive/Colab Notebooks/PROYECTO FINAL/Libros usando UnstructuredPDFLoader...


100%|██████████| 6/6 [41:41<00:00, 416.84s/it]


6 documentos correctamente cargados.


In [8]:
# Adicionar titulos
for doc in documents:
    ruta = doc.metadata.get("source", "")
    nombre_archivo = os.path.basename(ruta)           # e.g., "Revolucion_Francesa.pdf"
    titulo = os.path.splitext(nombre_archivo)[0]            # remove ".pdf"
    doc.metadata["title"] = titulo                    # now accessible as doc.metadata["title"]
print(f"\n Títulos de {len(documents)} documentos correctamente asignados.")


 Títulos de 6 documentos correctamente asignados.


### 3.2) Dividir documentos en bloques (chunks)

In [9]:
# Inicializar metodo para dividir los documentos en bloques
# - Tamaño de los bloques
chunk_size=1600
# - Tamaño de solapamiento entre bloques
chunk_overlap=400
# - Crear separador de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    add_start_index=True,
)
# Dividir los documentos en bloques
chunks = text_splitter.split_documents(documents)

## 3.3) Transformar los bloques de texto

Generar representaciones numéricas (embeddings) de fragmentos de texto utilizando un modelo de OpenAI y almacenarlas en una base de datos vectorial local llamada ChromaDB.

In [10]:
# Inicializar la herramienta para hacer el embeddings
modelo_embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENAI_API_KEY")
)

In [11]:
# Definir la ruta de vector embeddings
ruta_vector_embeddings = "chroma_ve"
# Usar chroma para crear los vector embeddings
print("Iniciando proceso para crear el vector embeddings...")
# Crear vector
print("Crear nuevo vector...")
vectorstore = Chroma(
    persist_directory=str(ruta_vector_embeddings),
    embedding_function=modelo_embedding
)
# Toca usarlo debido a la cantidad de archivos
batch_size = 100
# Agregar documentos
print(f"➕ Agregando {len(chunks)} nuevos documentos en lotes de {batch_size}...")
for i in tqdm(range(0, len(chunks), batch_size)):
    batch = chunks[i:i + batch_size]
    vectorstore.add_documents(batch)
print("\nNuevos documentos agregados y vectorstore actualizado.")

Iniciando proceso para crear el vector embeddings...
Crear nuevo vector...


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


➕ Agregando 3626 nuevos documentos en lotes de 100...


100%|██████████| 37/37 [01:04<00:00,  1.73s/it]


Nuevos documentos agregados y vectorstore actualizado.


In [13]:
# Cargar vector una vez creado
vectorstore = Chroma(
    persist_directory=str(ruta_vector_embeddings),
    embedding_function=modelo_embedding
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [14]:
# Revisar
try:
    num_embeddings = vectorstore._collection.count()
    print(f"Number of embeddings (documents/chunks) in the vector store: {num_embeddings}")
except AttributeError:
    print("Could not access _collection.count(). Ensure 'vectorstore' is a valid Chroma instance.")
except Exception as e:
    print(f"An error occurred while counting embeddings: {e}")

Number of embeddings (documents/chunks) in the vector store: 14504


## 4) Configurar LLM

In [15]:
# Parametros del LLM
# - Temperature:
#     Controla la aleatoriedad de las respuestas del modelo.
#     Un valor más bajo (cerca de 0) hace que el modelo sea más determinista,
#     Un valor más alto (hasta 1 o más) lo hace más creativo o variable.
temperature = 0.2
# - Max tokens
#     Define el número máximo de tokens que puede generar el modelo en una respuesta.
#     Cuantos más tokens, más larga puede ser la salida (pero consume más memoria y tiempo).
max_tokens=2000
# - Verbose
#   Indica si quieres ver mensajes detallados en la consola sobre el proceso de inferencia.
verbose=False

In [16]:
# Configuración LLM OpenAI
llm = ChatOpenAI(
    model_name="gpt-4", # o "gpt-3.5-turbo"
    temperature=temperature,
    max_tokens=max_tokens,
    api_key=os.getenv("OPENAI_API_KEY"),
    verbose=False
)

## 5) Crear el sistema RAG

### 5.1) Construir el prompt

In [17]:
# Plantilla del prompt usado en el sistema RAG
template = """
Eres un asistente de inteligencia artificial especializado en historia. Respondes preguntas únicamente utilizando el contexto documental proporcionado.

Reglas:
- No inventes información. Si el contexto no tiene la respuesta, dilo claramente.
- Sé específico y fundamenta tus respuestas citando fragmentos relevantes del contexto (usa comillas si es posible).
- Resume con claridad y precisión. Evita redundancias.
- No incluyas información externa al contexto.

Contexto:
{context}

Pregunta: {question}

Respuesta fundamentada únicamente en el contexto anterior:
"""
# Crear el prompt
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

### 5.2) Crear el pipeline del sistema RAG

In [18]:
# Configurar retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
# Pipeline de procesamiento del RAG
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

### 5.3) Crear funcion para que el usuario pregunte

In [19]:
# Funcion que tiene como entrada la pregunta o consulta del usuario.
def pregunta_usuario(pregunta):
    start_time = time.time()
    result = rag_pipeline.invoke({"query": pregunta})
    end_time = time.time()
    print(f"Pregunta: \n{pregunta}")
    print(f"Respuesta:")
    print(textwrap.fill(result['result'], width=80))
    print(f"Tiempo de computo: {end_time - start_time:.2f} segundos")
    print("\n\nDocumentos Fuente:")
    for doc in result["source_documents"]:
        titulo = doc.metadata.get("title", "Sin título")
        fragmento = doc.page_content.strip().replace("\n", " ")
        print(f"📄 {titulo} – Fragmento {i+1}:")
        print(f"\"{fragmento[:100]}...\"\n")

## 6) Test RAG

In [20]:
pregunta_usuario("¿Quién fue Fernando Raúl Klein?")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Pregunta: 
¿Quién fue Fernando Raúl Klein?
Respuesta:
Fernando Raúl Klein es el autor de la tesis titulada "Historia y memoria de la
inmigración judía sefardí al Uruguay. Análisis de sus prácticas sociales y modos
de inserción en la sociedad uruguaya. 1908-1937". Esta tesis fue presentada para
la obtención del grado de Doctor en Historia en la Universidad Nacional de La
Plata.
Tiempo de computo: 4.60 segundos


Documentos Fuente:
📄 Sin título – Fragmento 3601:
"Klein, Fernando Raúl  Historia y memoria de lainmigración judía sefardí alUruguay  Tesis presentada ..."

📄 Historia y memoria de la inmigración judía sefardí al Uruguay – Fragmento 3601:
"Klein, Fernando Raúl  Historia y memoria de lainmigración judía sefardí alUruguay  Tesis presentada ..."

📄 Historia y memoria de la inmigración judía sefardí al Uruguay – Fragmento 3601:
"Klein, Fernando Raúl  Historia y memoria de lainmigración judía sefardí alUruguay  Tesis presentada ..."

📄 Sin título – Fragmento 3601:
"Klein, Ferna

In [21]:
pregunta_usuario('¿Cual fue la distribucion regional de la emigracion española hacia latinoamerica en los anos 1965-70?')

Pregunta: 
¿Cual fue la distribucion regional de la emigracion española hacia latinoamerica en los anos 1965-70?
Respuesta:
El contexto no proporciona información específica sobre la distribución regional
de la emigración española hacia Latinoamérica en los años 1965-70.
Tiempo de computo: 4.20 segundos


Documentos Fuente:
📄 Sin título – Fragmento 3601:
"211  FIGURA 1. Evolución anual de la emigración española asistida a Latinoamérica, 1968-1990.  CUADR..."

📄 Sin título – Fragmento 3601:
"211  FIGURA 1. Evolución anual de la emigración española asistida a Latinoamérica, 1968-1990.  CUADR..."

📄 la-emigracin-espaola-asistida-a-latinoamrica-19681990-0 – Fragmento 3601:
"211  FIGURA 1. Evolución anual de la emigración española asistida a Latinoamérica, 1968-1990.  CUADR..."

📄 la-emigracin-espaola-asistida-a-latinoamrica-19681990-0 – Fragmento 3601:
"211  FIGURA 1. Evolución anual de la emigración española asistida a Latinoamérica, 1968-1990.  CUADR..."

📄 Sin título – Fragmento 3601:
"

### 7) Descargar vector database

In [22]:
import shutil
from google.colab import files
# Comprimir en un archivo ZIP
shutil.make_archive("chroma_ve", 'zip', ruta_vector_embeddings)
# Descargar embedings vector database
files.download("chroma_ve.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>